In [7]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException,ElementNotInteractableException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import random

options = webdriver.ChromeOptions()
# options.add_argument('--disable-blink-features=AutomationControlled')  # Disable automation flags
options.add_experimental_option('excludeSwitches', ['enable-logging'])
options.add_experimental_option(
        "prefs", {
            # block image loading
            "profile.managed_default_content_settings.images": 2,
        }
    )
# options.add_argument('--headless')  # Uncomment for headless mode

service = Service(ChromeDriverManager().install())

# driver = webdriver.Chrome(service=service, options=options)
# wait = WebDriverWait(driver, 10)
# driver.get("https://alonhadat.com.vn/can-ban-nha-da-nang-t3/trang-200.htm")


In [8]:
data_list = []
STANDARD_KEYS = ['price','area', 'Mã tin','Hướng','Phòng ăn', 'Loại tin','Đường trước nhà',
                 'Nhà bếp','Loại BDS', 'Pháp lý', 'Sân thượng', 'Chiều ngang', 'Số lầu',
                 'Chổ để xe hơi','Chiều dài','Số phòng ngủ','Chính chủ']

## Supporting function delay and scroll to bottom of page(avoid bot detection)

In [9]:

def human_like_delay(min_time=1, max_time=3):
    time.sleep(random.uniform(min_time, max_time))

def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    human_like_delay(1, 3)


## Scrape on the main page function (get all thumbnails clickable to access each card)

In [10]:

def scrape_main_page(url):
    driver.get(url)
    scroll_to_bottom()
    human_like_delay(2, 4)

    # Wait for the presence of elements with class 'thumbnail'
    wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'thumbnail')))
    
    # Locate the thumbnail elements (a tags within div with class 'thumbnail')
    thumbnails = driver.find_elements(By.XPATH, '//div[@class="thumbnail"]/a')

    for i in range(len(thumbnails)):
        try:
            print(f"Clicking on thumbnail {i + 1}...")

            # Scroll to the thumbnail and click it
            driver.execute_script("arguments[0].scrollIntoView();", thumbnails[i])
            human_like_delay(1, 2)
            thumbnails[i].click()  # Click the thumbnail

            human_like_delay(1, 3)

            # Scrape the data on the clicked page
            scrape_page()
            
            human_like_delay(1, 4)

        except Exception as e:
            print(f"An error occurred while clicking on thumbnail {i + 1}: {e}")
        
        finally:
            # Go back to the main page after scraping
            try:
                driver.back()
                human_like_delay(2, 3)
            except:
                driver.get(url)  # Reload the main page if back fails

## Get card's data function(using XPATH)

In [11]:
def scrape_page():
    data_dict = {}
    human_like_delay(1,3)
    price = driver.find_element(By.XPATH, '//div[@class="moreinfor"]//span[@class="price"]//span[@class="value"]').text.strip()
    area = driver.find_element(By.XPATH, '//div[@class="moreinfor"]//span[@class="square"]//span[@class="value"]').text.strip()
    data_dict['price'] = price
    data_dict['area'] = area
    human_like_delay(2,4)
    info = driver.find_elements(By.XPATH, '//div[@class="moreinfor1"]//div[@class="infor"]/table/tbody/tr/td')
    for i, li in enumerate(info):
        try:
            if i % 2 == 0:
                key = li.text.strip()  
            else:
                try:
                    if li.find_element(By.TAG_NAME, 'img'):
                        val = 1
                except:
                    if li.text.strip() == '---':
                        val = None 
                    else:
                        val = li.text.strip() 
                
                data_dict[key] = val
        except (NoSuchElementException, StaleElementReferenceException) as e:
            print(f"Error processing element: {e}")
            continue

    for key in STANDARD_KEYS:
        if key not in data_dict:
            data_dict[key] = None
    data_list.append(data_dict)
    print(data_list)

## Checkpoint

In [ ]:
datalis = []

In [13]:
driver = webdriver.Chrome(service=service, options=options)
wait = WebDriverWait(driver, 10)
try:
    for i in range(6,50):
        try:
            url = f'https://alonhadat.com.vn/can-ban-nha-da-nang-t3/trang-{i}.htm'
            scrape_main_page(url)
        except:
            break
except Exception:
    print("An error occurred while scraping the main page.")
finally:
    print(data_list)
    df = pd.DataFrame(data_list)
    df.to_csv('scraped_data_alonhadat.csv',encoding="utf-8", index=False)
    print("Saved successfully")
# driver.quit()


Clicking on thumbnail 1...
[{'price': '5,19 tỷ', 'area': '94 m\n2', 'Mã tin': '15155012', 'Hướng': '_', 'Phòng ăn': 1, 'Loại tin': 'Cần bán', 'Đường trước nhà': '7,5m', 'Nhà bếp': 1, 'Loại BDS': 'Nhà mặt tiền', 'Pháp lý': 'Sổ hồng/ Sổ đỏ', 'Sân thượng': 1, 'Chiều ngang': '6m', 'Số lầu': '2', 'Chổ để xe hơi': 1, 'Chiều dài': None, 'Số phòng ngủ': '6', 'Chính chủ': None}]
Clicking on thumbnail 2...
[{'price': '5,19 tỷ', 'area': '94 m\n2', 'Mã tin': '15155012', 'Hướng': '_', 'Phòng ăn': 1, 'Loại tin': 'Cần bán', 'Đường trước nhà': '7,5m', 'Nhà bếp': 1, 'Loại BDS': 'Nhà mặt tiền', 'Pháp lý': 'Sổ hồng/ Sổ đỏ', 'Sân thượng': 1, 'Chiều ngang': '6m', 'Số lầu': '2', 'Chổ để xe hơi': 1, 'Chiều dài': None, 'Số phòng ngủ': '6', 'Chính chủ': None}, {'price': '11 tỷ', 'area': '90 m\n2', 'Mã tin': '15166135', 'Hướng': 'Đông', 'Phòng ăn': 1, 'Loại tin': 'Cần bán', 'Đường trước nhà': '10m', 'Nhà bếp': 1, 'Loại BDS': 'Nhà mặt tiền', 'Pháp lý': 'Sổ hồng/ Sổ đỏ', 'Sân thượng': 1, 'Chiều ngang': '4,5m', 'S